Load ratings and books datasets, then merge them based on 'book_id'.

In [ ]:
import pandas as pd

ratings = pd.read_csv('ratings.csv')
books = pd.read_csv('books.csv')
df = pd.merge(ratings, books, on='book_id')

Create a copy of the merged DataFrame for training purposes.

In [ ]:
train_df = df.copy()

Pivot the DataFrame to create a user-item rating matrix.

In [ ]:
user_ratings = train_df.pivot(index='user_id', columns='book_id', values='rating')

Create a Surprise dataset from the training DataFrame.

In [ ]:
from surprise import Dataset, Reader
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(train_df[['user_id', 'book_id', 'rating']], reader)

Prepare a train/test split from the Surprise dataset.

In [ ]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.2)

Train a KNN user-based model using the training set.

In [ ]:
from surprise import KNNBasic
user_based_model = KNNBasic(similarity='cosine')
user_based_model.fit(trainset)

Predict and evaluate the user-based model using RMSE.

In [ ]:
from surprise import accuracy
predictions = user_based_model.test(testset)
accuracy.rmse(predictions)

Train a KNN item-based model using the training set.

In [ ]:
item_based_model = KNNBasic(similarity='cosine', user_based=False)
item_based_model.fit(trainset)

Predict and evaluate the item-based model using RMSE.

In [ ]:
predictions_item = item_based_model.test(testset)
accuracy.rmse(predictions_item)

Define a function to generate book recommendations for a given user.

In [ ]:
def generate_recommendations(user_id, model, n=5):
    user_inner_id = trainset.to_inner_uid(user_id)
    user_ratings = model.get_neighbors(user_inner_id, k=n)
    return [trainset.to_raw_iid(i) for i in user_ratings]